In [2]:
import pandas as pd
import seaborn as sns
import copy
import matplotlib
import plotly.express as px
from PIL import Image
import matplotlib.pyplot as plt
from pylab import rcParams
# Reglas de Asociación
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

from sklearn.metrics import accuracy_score
from sklearn import metrics
from sklearn.datasets import make_blobs

from joblib import dump, load
from sklearn.preprocessing import StandardScaler

# Análisis

In [3]:
df_items = pd.read_csv("https://raw.githubusercontent.com/pokengineer/DataScience/main/datasets/coffee_items.csv")
df_orders = pd.read_csv("https://raw.githubusercontent.com/pokengineer/DataScience/main/datasets/coffee_orders.csv")

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [4]:
df_items.head()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,item_id,sku,item_name,item_cat,item_size,item_price
0,It001,HDR-CAP-MD,Cappuccino,Hot Drinks,Medium,3.45
1,It002,HDR-CAP-LG,Cappuccino,Hot Drinks,Large,3.75
2,It003,HDR-LAT-MD,Latte,Hot Drinks,Medium,3.45
3,It004,HDR-LAT-LG,Latte,Hot Drinks,Large,3.75
4,It005,HDR-FLT,Flat White,Hot Drinks,NaN,3.15


In [5]:
df_orders.head()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,row_id,order_id,created_at,item_id,quantity,cust_name,in_or_out
0,1,ORD001,2024-02-12 07:04:19,It008,1,Alex,out
1,2,ORD002,2024-02-12 07:09:38,It014,1,Jordan,in
2,3,ORD003,2024-02-12 07:14:29,It008,1,Taylor,out
3,4,ORD004,2024-02-12 07:18:39,It019,1,Casey,out
4,5,ORD005,2024-02-12 07:23:44,It024,1,Jamie,out


In [6]:
print(df_orders.shape[0])

521


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [7]:
df_orders.order_id.duplicated().sum()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


91

In [8]:
df_orders[df_orders['quantity']<=0]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,row_id,order_id,created_at,item_id,quantity,cust_name,in_or_out


In [ ]:
df_orders[df_orders['order_id']=='ORD029']

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,row_id,order_id,created_at,item_id,quantity,cust_name,in_or_out
38,39,ORD029,2024-02-12 09:19:11,It005,1,Riley,in
39,40,ORD029,2024-02-12 09:19:11,It020,1,Riley,NaN
40,41,ORD029,2024-02-12 09:19:11,It012,1,Riley,NaN


# columna de descripción

In [9]:
df_items['descripcion'] =df_items.apply(lambda x: str(x['item_name']) + ' | ' + str(x['item_size']), axis=1)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [10]:
df_items.head(3)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,item_id,sku,item_name,item_cat,item_size,item_price,descripcion
0,It001,HDR-CAP-MD,Cappuccino,Hot Drinks,Medium,3.45,Cappuccino | Medium
1,It002,HDR-CAP-LG,Cappuccino,Hot Drinks,Large,3.75,Cappuccino | Large
2,It003,HDR-LAT-MD,Latte,Hot Drinks,Medium,3.45,Latte | Medium


In [11]:
df_orders.head(3)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,row_id,order_id,created_at,item_id,quantity,cust_name,in_or_out
0,1,ORD001,2024-02-12 07:04:19,It008,1,Alex,out
1,2,ORD002,2024-02-12 07:09:38,It014,1,Jordan,in
2,3,ORD003,2024-02-12 07:14:29,It008,1,Taylor,out


In [12]:
df_orders = pd.merge(df_orders,df_items[['item_id','descripcion']],on='item_id',how='left')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# reglas de asociación

In [13]:
# Quitamos los ítems DOTCOM POSTAGE y POSTAGE porque no son productos.
# Llenamos con 0 los pedidos donde un producto no fué comprado
df_group = (df_orders.groupby(['order_id', 'descripcion'])['quantity'].sum().unstack().reset_index().fillna(0).set_index('order_id'))
# Seteamos True/False dependiendo de cada valor
df_group = df_group.applymap(lambda x: True if x >0 else False)
df_group

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


descripcion,Cappuccino | Large,Cappuccino | Medium,Caramel Macchiato | Large,Caramel Macchiato | Medium,Cold Coffee | Large,Cold Coffee | Medium,Cold Mocha | Large,Cold Mocha | Medium,Espresso | nan,Flat White | nan,...,Latte | Large,Latte | Medium,Lemonade | Large,Lemonade | Medium,Mocha | Large,Mocha | Medium,Sandwich Ham&Cheese | nan,Sandwich Salami&Mozzarella | nan,White Mocha | Large,White Mocha | Medium
order_id,,,,,,,,,,,,,,,,,,,,,
ORD001,False,False,False,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
ORD002,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
ORD003,False,False,False,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
ORD004,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
ORD005,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ORD432,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False
ORD433,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False
ORD434,False,False,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [14]:
x = 0.005
frequent_itemsets = apriori(df_group, min_support=x, use_colnames=True)
frequent_itemsets.sort_values(by="support", ascending=False)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,support,itemsets
6,0.069409,(Cold Mocha | Large)
9,0.069409,(Flat White | nan)
19,0.066838,(Mocha | Medium)
10,0.056555,(Hot Chocolate | Large)
23,0.056555,(White Mocha | Medium)
1,0.053985,(Cappuccino | Medium)
12,0.053985,(Iced Tea | Large)
16,0.053985,(Lemonade | Large)
22,0.051414,(White Mocha | Large)
3,0.051414,(Caramel Macchiato | Medium)


In [15]:
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
# Ordenamos por confianza de mayor a menor
rules.sort_values(by="confidence", ascending=False).head(5)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
6,(Lemonade | Large),(Cold Mocha | Large),0.053985,0.069409,0.012853,0.238095,3.430335,0.009106,1.221401,0.748913
23,(Sandwich Ham&Cheese | nan),(White Mocha | Medium),0.041131,0.056555,0.007712,0.187500,3.315341,0.005386,1.161163,0.728329
7,(Cold Mocha | Large),(Lemonade | Large),0.069409,0.053985,0.012853,0.185185,3.430335,0.009106,1.161019,0.761326
13,(Sandwich Salami&Mozzarella | nan),(Espresso | nan),0.048843,0.048843,0.007712,0.157895,3.232687,0.005326,1.129499,0.726126
12,(Espresso | nan),(Sandwich Salami&Mozzarella | nan),0.048843,0.048843,0.007712,0.157895,3.232687,0.005326,1.129499,0.726126


In [16]:
rules

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(Cappuccino | Medium),(Cold Coffee | Large),0.053985,0.046272,0.005141,0.095238,2.058201,0.002643,1.054120,0.543478
1,(Cold Coffee | Large),(Cappuccino | Medium),0.046272,0.053985,0.005141,0.111111,2.058201,0.002643,1.064267,0.539084
2,(Caramel Macchiato | Medium),(Cold Mocha | Large),0.051414,0.069409,0.005141,0.100000,1.440741,0.001573,1.033990,0.322493
3,(Cold Mocha | Large),(Caramel Macchiato | Medium),0.069409,0.051414,0.005141,0.074074,1.440741,0.001573,1.024473,0.328729
4,(Espresso | nan),(Cold Coffee | Large),0.048843,0.046272,0.005141,0.105263,2.274854,0.002881,1.065931,0.589189
5,(Cold Coffee | Large),(Espresso | nan),0.046272,0.048843,0.005141,0.111111,2.274854,0.002881,1.070051,0.587601
6,(Lemonade | Large),(Cold Mocha | Large),0.053985,0.069409,0.012853,0.238095,3.430335,0.009106,1.221401,0.748913
7,(Cold Mocha | Large),(Lemonade | Large),0.069409,0.053985,0.012853,0.185185,3.430335,0.009106,1.161019,0.761326
8,(Lemonade | Medium),(Cold Mocha | Medium),0.048843,0.051414,0.005141,0.105263,2.047368,0.002630,1.060184,0.537838
9,(Cold Mocha | Medium),(Lemonade | Medium),0.051414,0.048843,0.005141,0.100000,2.047368,0.002630,1.056841,0.539295
